In [1]:
import os
import pandas as pd
import numpy as np
import torch
import cv2
import shutil
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
#data_path = "../SEWA/SEWAv02"

In [61]:
'''vgg16 = models.vgg16(pretrained=True)
vgg16 = nn.Sequential(*list(vgg16.features.children()))

def extract_features_pytorch(image_path):
    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = preprocess(img).unsqueeze(0)  # Add batch dimension
    
    # Pass the image through the VGG16 model
    features = vgg16(img)
    
    return features.squeeze().detach().numpy()

def extract_features_from_frames_pytorch(frames_dir):
    frame_features = {}
    # Iterate through all frames in the directory
    for filename in os.listdir(frames_dir):
        if filename.endswith('.jpg'):
            # Extract features from the frame using VGG
            frame_path = os.path.join(frames_dir, filename)
            features = extract_features_pytorch(frame_path)
            frame_features[filename] = features
    return frame_features'''

In [ ]:
'''save_path = './vgg16'
for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)
    vid_file = [f for f in os.listdir(file_path) if f.endswith('.avi')]
    
    if vid_file:
        vid_name = vid_file[0].split('.')[0]  # Extract video name without extension
        vid_dir = os.path.join(save_path, vid_name)  # Directory for saving features
        
        # Create output directory for frames
        output_dir = 'test_jpg'
        os.makedirs(output_dir, exist_ok=True)
        
        # Open video file
        vid_file_path = os.path.join(file_path, vid_file[0])
        cap = cv2.VideoCapture(vid_file_path)
        
        frame_count = 1
        desired_duration = 15
        frame_rate = cap.get(cv2.CAP_PROP_FPS)
        
        # Iterate over frames and extract features
        while cap.isOpened() and frame_count <= frame_rate * desired_duration:  
            ret, frame = cap.read()
            if not ret:
                break
            
            # Save the frame as an image in the output directory
            frame_path = os.path.join(output_dir, f'frame_{frame_count}.jpg')
            cv2.imwrite(frame_path, frame)
            
            frame_count += 1
        
        cap.release()
        
        # Extract features from frames
        #print('Extracted Frames:', frame_count)
        features = extract_features_from_frames_pytorch(output_dir)
        # Create directory for video features
        os.makedirs(vid_dir, exist_ok=True)
        #print(features)
        # Save features as numpy arrays
        for idx in features:
            frame = idx.split('.')[0]
            p = os.path.join(vid_dir, f'{frame}.npy')
            print(p)
            np.save(p, features[idx])
        
        # Remove temporary directory
        shutil.rmtree(output_dir)'''

In [30]:
#features = extract_features_from_frames_pytorch(output_dir)

In [23]:
'''def extract_vgg_features(image_path, layer_name='block4_pool'):
    base_model = VGG16(weights='imagenet', include_top=False)
    # Create feature extractor model
    model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    # Extract features
    features = model.predict(img)
    return features

# Define a function to extract features from all frames in a directory
def extract_features_from_frames(frames_dir):
    frame_features = {}
    # Iterate through all frames in the directory
    for filename in os.listdir(frames_dir):
        if filename.endswith('.jpg'):
            # Extract features from the frame using VGG
            frame_path = os.path.join(frames_dir, filename)
            features = extract_vgg_features(frame_path)
            frame_features[filename] = features
            break
    return frame_features'''

In [2]:
def max_pool_vgg_features(arr):
    batch_size, seqlen = 1, 512
    x = torch.randn(batch_size, seqlen, 7, 7)
    # Applying max pooling operation along the 7x7 dimensions
    pooled_output, _ = torch.max(x.view(batch_size, seqlen, -1), dim=-1)
    return pooled_output

In [3]:
def extract_val_and_arou(sewa, file_name):
    folder = file_name
    folder_path = os.path.join(sewa, folder)
    arou_file = folder_path + '/' + file_name + '_Arousal_V_Aligned.csv'
    val_file = folder_path +  '/' + file_name + '_Valence_V_Aligned.csv'
    print(val_file)
    #print(folder_path + '/' + arou_file[0])
    df_arou = pd.read_csv(arou_file)
    df_val = pd.read_csv(val_file)
    return df_arou, df_val

In [77]:
'''path = '/home1/shrirama/project/Cross-Cultural-Emotion-Recognition/vgg16'
save_path = '/home1/shrirama/project/Cross-Cultural-Emotion-Recognition/vggpooled/'
#project/vgg16/SAL_C6_S185_P370_VC1_006763_007716/frame_657.npy
data = []
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    pooled_list = []
    for frame in os.listdir(file_path):
        framepath = os.path.join(file_path, frame)
        arr = np.load(framepath)
        pool = max_pool_vgg_features(arr)
        pooled_list.append(pool)
    final_tensor = torch.cat(pooled_list, dim=0)
    tensor_name = save_path + file + '.pt'
    torch.save(final_tensor, tensor_name)'''

In [18]:
'''SEWA_path = "../SEWA/SEWAv02"
label_tensors_valence = []
label_tensors_arousal = []
for folder in os.listdir(SEWA_path):
    file_path = os.path.join(SEWA_path, folder)
    for file in os.listdir(file_path):
        if file.endswith('_Arousal_V_Aligned.csv'):
            arousal_df = pd.read_csv(os.path.join(file_path, file))
            arousal_values = arousal_df['arousal'].values[:750]  # Take first 750 frames
            padded_arousal_values = np.pad(arousal_values, (0, max_length - len(arousal_values)), mode='constant')
            label_tensors_arousal.append(padded_arousal_values)
        elif file.endswith('_Valence_V_Aligned.csv'):
            valence_df = pd.read_csv(os.path.join(file_path, file))
            valence_values = valence_df['valence'].values[:750]  # Take first 750 frames
            padded_valence_values = np.pad(valence_values, (0, max_length - len(valence_values)), mode='constant')
            label_tensors_valence.append(padded_valence_values)'''

'SEWA_path = "../SEWA/SEWAv02"\nlabel_tensors_valence = []\nlabel_tensors_arousal = []\nfor folder in os.listdir(SEWA_path):\n    file_path = os.path.join(SEWA_path, folder)\n    for file in os.listdir(file_path):\n        if file.endswith(\'_Arousal_V_Aligned.csv\'):\n            arousal_df = pd.read_csv(os.path.join(file_path, file))\n            arousal_values = arousal_df[\'arousal\'].values[:750]  # Take first 750 frames\n            padded_arousal_values = np.pad(arousal_values, (0, max_length - len(arousal_values)), mode=\'constant\')\n            label_tensors_arousal.append(padded_arousal_values)\n        elif file.endswith(\'_Valence_V_Aligned.csv\'):\n            valence_df = pd.read_csv(os.path.join(file_path, file))\n            valence_values = valence_df[\'valence\'].values[:750]  # Take first 750 frames\n            padded_valence_values = np.pad(valence_values, (0, max_length - len(valence_values)), mode=\'constant\')\n            label_tensors_valence.append(padded_va

In [5]:
save_path = 'vggpooled/'
SEWA_path = "../SEWAv02"
max_length = 750
feature_tensors = []
label_tensors_valence = []
label_tensors_arousal = []
for file in os.listdir(save_path):
    #Extract tensors
    file_path = os.path.join(save_path, file)
    file_name = file[:-3]
    arr = torch.load(file_path)
    feature_tensors.append(arr)
    
    #Extract Valence and Arousal
    label_file_path = os.path.join(SEWA_path, file_name)
    #print(label_file_path)
    for label_file in os.listdir(label_file_path):
        #print(label_file)
        if label_file.endswith('_Arousal_V_Aligned.csv'):
            #print('Here')
            arousal_df = pd.read_csv(os.path.join(label_file_path, label_file))
            arousal_values = arousal_df['arousal'].values[:750]  # Take first 750 frames
            padded_arousal_values = np.pad(arousal_values, (0, max_length - len(arousal_values)), mode='constant')
            label_tensors_arousal.append(padded_arousal_values)
        elif label_file.endswith('_Valence_V_Aligned.csv'):
            #print('Here')
            valence_df = pd.read_csv(os.path.join(label_file_path, label_file))
            valence_values = valence_df['valence'].values[:750]  # Take first 750 frames
            padded_valence_values = np.pad(valence_values, (0, max_length - len(valence_values)), mode='constant')
            label_tensors_valence.append(padded_valence_values)
    

In [6]:
padded_features,mask = pad_sequences(feature_tensors, maxlen=max_length, padding='post', dtype='float32')
X_train, X_test = train_test_split(padded_features, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (332, 750, 512)
X_test shape: (84, 750, 512)


In [7]:
normalized_feature_tensors = []
for video_features in padded_features:
    # Compute mean and standard deviation along the first axis (frames)
    mean = np.mean(video_features, axis=0, keepdims=True)
    std = np.std(video_features, axis=0, keepdims=True)
    # Apply normalization to each frame
    normalized_video_features = (video_features - mean) / (std + 1e-8)  # Adding a small epsilon to avoid division by zero
    normalized_feature_tensors.append(normalized_video_features)

# Convert the list of normalized feature tensors back to a numpy array
normalized_feature_tensors = np.array(normalized_feature_tensors)

In [8]:
X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (332, 750, 512)
X_test shape: (84, 750, 512)


In [9]:
y_valence = np.array(label_tensors_valence)
y_arousal = np.array(label_tensors_arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

y_train_valence shape: (332, 750)
y_test_valence shape: (84, 750)
y_train_arousal shape: (332, 750)
y_test_arousal shape: (84, 750)


In [10]:
class BasicNN(nn.Module):
    def __init__(self):
        super(BasicNN, self).__init__()
        self.fc1 = nn.Linear(750 * 512, 512)  # Assuming input features have shape (750, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3_valence = nn.Linear(256, 750)  # Output layer for valence prediction
        self.fc3_arousal = nn.Linear(256, 750)  # Output layer for arousal prediction
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 750 * 512)  # Flatten input features
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        valence_output = self.fc3_valence(x)
        arousal_output = self.fc3_arousal(x)
        return valence_output, arousal_output

In [ ]:
##GXY
class RmseLoss(nn.Module):
    def __init__(self):
        super(RmseLoss, self).__init__()
    def forward(self,  masked_outputs, masked_labels,effective_lengths):
        criterion = nn.MSELoss(reduction='none')
        loss = criterion(masked_outputs, masked_labels)
        loss = torch.sum(loss, dim=1) / effective_lengths
        loss = torch.mean(loss)
        loss = torch.sqrt(loss)
        return loss

In [11]:
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_valence_tensor = torch.FloatTensor(y_train_valence)
y_test_valence_tensor = torch.FloatTensor(y_test_valence)
y_train_arousal_tensor = torch.FloatTensor(y_train_arousal)
y_test_arousal_tensor = torch.FloatTensor(y_test_arousal)

train_dataset = TensorDataset(X_train_tensor, y_train_valence_tensor, y_train_arousal_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_valence_tensor, y_test_arousal_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

model = BasicNN()
# criterion = nn.MSELoss()  # Mean Squared Error loss
##GXY
criterion = RmseLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_features, batch_valence_labels, batch_arousal_labels in train_loader:
        optimizer.zero_grad()
        valence_output, arousal_output = model(batch_features)

        ## GXY
        mask_valence_output=valence_output* mask.float()
        mask_arousal_output=arousal_output* mask.float()
        effective_lengths = torch.sum(mask, dim=1)
        valence_loss = criterion(mask_valence_output, batch_valence_labels,effective_lengths)
        arousal_loss = criterion(mask_valence_output, batch_arousal_labels,effective_lengths)

        
        loss = valence_loss + arousal_loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_features.size(0)
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

    # Evaluation loop
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for batch_features, batch_valence_labels, batch_arousal_labels in test_loader:
            valence_output, arousal_output = model(batch_features)
            valence_loss = criterion(valence_output, batch_valence_labels)
            arousal_loss = criterion(arousal_output, batch_arousal_labels)
            loss = valence_loss + arousal_loss
            test_loss += loss.item() * batch_features.size(0)
        test_loss /= len(test_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}")

Epoch 1/10, Train Loss: 18.2844
Epoch 1/10, Test Loss: 0.7919
Epoch 2/10, Train Loss: 17.4159
Epoch 2/10, Test Loss: 0.4068


KeyboardInterrupt: 

In [107]:
class BasicRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BasicRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc_valence = nn.Linear(hidden_size, output_size)
        self.fc_arousal = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        valence_output = self.fc_valence(out[:, -1, :])  # Get output from last time step
        arousal_output = self.fc_arousal(out[:, -1, :])  # Get output from last time step
        return valence_output, arousal_output

class CCCLoss(nn.Module):
    def __init__(self):
        super(CCCLoss, self).__init__()

    def forward(self, y_pred, y_true):
        mean_pred = torch.mean(y_pred, dim=1, keepdim=True)
        mean_true = torch.mean(y_true, dim=1, keepdim=True)
        cov_pred = torch.sum((y_pred - mean_pred) * (y_true - mean_true), dim=1, keepdim=True)
        var_pred = torch.sum((y_pred - mean_pred) ** 2, dim=1, keepdim=True)
        var_true = torch.sum((y_true - mean_true) ** 2, dim=1, keepdim=True)
        ccc = 2 * cov_pred / (var_pred + var_true + (mean_pred - mean_true) ** 2 + 1e-8)  # Adding a small epsilon to avoid division by zero
        return 1 - torch.mean(ccc)

In [108]:
def RNN_model(X_train, X_test):

    # Hyperparameters
    input_size = 512  # Assuming each frame is represented by a feature vector of size 512
    hidden_size = 128
    num_layers = 4
    output_size = 750  # Number of frames
    
    X_train_tensor = torch.FloatTensor(X_train)
    X_test_tensor = torch.FloatTensor(X_test)
    y_train_valence_tensor = torch.FloatTensor(y_train_valence)
    y_test_valence_tensor = torch.FloatTensor(y_test_valence)
    y_train_arousal_tensor = torch.FloatTensor(y_train_arousal)
    y_test_arousal_tensor = torch.FloatTensor(y_test_arousal)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_valence_tensor, y_train_arousal_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_valence_tensor, y_test_arousal_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)
    
    model = BasicRNN(input_size, hidden_size, num_layers, output_size)
    criterion = nn.MSELoss()  # Mean Squared Error loss
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        valence_train_loss = 0.0
        arousal_train_loss = 0.0
        
        for batch_features, batch_valence_labels, batch_arousal_labels in train_loader:
            optimizer.zero_grad()
            valence_output, arousal_output = model(batch_features)
            
            # Compute separate losses for valence and arousal
            valence_loss = criterion(valence_output, batch_valence_labels)
            arousal_loss = criterion(arousal_output, batch_arousal_labels)
            
            # Accumulate separate losses
            valence_train_loss += valence_loss.item() * batch_features.size(0)
            arousal_train_loss += arousal_loss.item() * batch_features.size(0)
            
            # Backpropagation and optimization
            loss = valence_loss + arousal_loss
            loss.backward()
            optimizer.step()
        
        # Average losses over the entire dataset
        valence_train_loss /= len(train_loader.dataset)
        arousal_train_loss /= len(train_loader.dataset)
        
        # Print separate losses for valence and arousal
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Train Loss: {valence_train_loss:.4f}, Arousal Train Loss: {arousal_train_loss:.4f}")
    
        # Evaluation loop
        model.eval()
        valence_test_loss = 0.0
        arousal_test_loss = 0.0
        
        with torch.no_grad():
            for batch_features, batch_valence_labels, batch_arousal_labels in test_loader:
                valence_output, arousal_output = model(batch_features)
                
                # Compute separate losses for valence and arousal
                valence_loss = criterion(valence_output, batch_valence_labels)
                arousal_loss = criterion(arousal_output, batch_arousal_labels)
                
                # Accumulate separate losses
                valence_test_loss += valence_loss.item() * batch_features.size(0)
                arousal_test_loss += arousal_loss.item() * batch_features.size(0)
                
            # Average losses over the entire dataset
            valence_test_loss /= len(test_loader.dataset)
            arousal_test_loss /= len(test_loader.dataset)
        
        # Print separate losses for valence and arousal in the test set
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Test Loss: {valence_test_loss:.4f}, Arousal Test Loss: {arousal_test_loss:.4f}")

In [38]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1738, Arousal Train Loss: 0.1659
Epoch 1/50, Valence Test Loss: 0.1212, Arousal Test Loss: 0.1080
Epoch 2/50, Valence Train Loss: 0.1278, Arousal Train Loss: 0.1319
Epoch 2/50, Valence Test Loss: 0.1136, Arousal Test Loss: 0.1069
Epoch 3/50, Valence Train Loss: 0.1045, Arousal Train Loss: 0.1030
Epoch 3/50, Valence Test Loss: 0.1317, Arousal Test Loss: 0.1155
Epoch 4/50, Valence Train Loss: 0.0923, Arousal Train Loss: 0.0910
Epoch 4/50, Valence Test Loss: 0.1218, Arousal Test Loss: 0.1052
Epoch 5/50, Valence Train Loss: 0.0879, Arousal Train Loss: 0.0802
Epoch 5/50, Valence Test Loss: 0.1408, Arousal Test Loss: 0.1230
Epoch 6/50, Valence Train Loss: 0.0877, Arousal Train Loss: 0.0792
Epoch 6/50, Valence Test Loss: 0.1197, Arousal Test Loss: 0.1108
Epoch 7/50, Valence Train Loss: 0.0815, Arousal Train Loss: 0.0719
Epoch 7/50, Valence Test Loss: 0.1178, Arousal Test Loss: 0.1163
Epoch 8/50, Valence Train Loss: 0.0767, Arousal Train Loss: 0.0693
Epoch 8/5

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_valence = nn.Linear(hidden_size, output_size)
        self.fc_arousal = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)
        valence_output = self.fc_valence(out[:, -1, :])  # Get output from last time step
        arousal_output = self.fc_arousal(out[:, -1, :])  # Get output from last time step
        return valence_output, arousal_output

In [12]:
def LSTM_model(X_train, X_test):

    input_size = 512  # Assuming each frame is represented by a feature vector of size 512
    hidden_size = 256
    num_layers = 4
    output_size = 750  # Number of frames
    dropout = 0.5
    
    # Convert NumPy arrays to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    X_test_tensor = torch.FloatTensor(X_test)
    y_train_valence_tensor = torch.FloatTensor(y_train_valence)
    y_test_valence_tensor = torch.FloatTensor(y_test_valence)
    y_train_arousal_tensor = torch.FloatTensor(y_train_arousal)
    y_test_arousal_tensor = torch.FloatTensor(y_test_arousal)
    
    # Create DataLoader for training and testing sets
    train_dataset = TensorDataset(X_train_tensor, y_train_valence_tensor, y_train_arousal_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_valence_tensor, y_test_arousal_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)
    
    model = LSTM(input_size, hidden_size, num_layers, output_size)
    criterion = nn.MSELoss()  # Mean Squared Error loss
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        valence_train_loss = 0.0
        arousal_train_loss = 0.0
        
        for batch_features, batch_valence_labels, batch_arousal_labels in train_loader:
            optimizer.zero_grad()
            valence_output, arousal_output = model(batch_features)
            
            # Compute separate losses for valence and arousal
            valence_loss = criterion(valence_output, batch_valence_labels)
            arousal_loss = criterion(arousal_output, batch_arousal_labels)
            
            # Accumulate separate losses
            valence_train_loss += valence_loss.item() * batch_features.size(0)
            arousal_train_loss += arousal_loss.item() * batch_features.size(0)
            
            # Backpropagation and optimization
            loss = valence_loss + arousal_loss
            loss.backward()
            optimizer.step()
        
        # Average losses over the entire dataset
        valence_train_loss /= len(train_loader.dataset)
        arousal_train_loss /= len(train_loader.dataset)
        
        # Print separate losses for valence and arousal
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Train Loss: {valence_train_loss:.4f}, Arousal Train Loss: {arousal_train_loss:.4f}")
    
        # Evaluation loop
        model.eval()
        valence_test_loss = 0.0
        arousal_test_loss = 0.0
        
        with torch.no_grad():
            for batch_features, batch_valence_labels, batch_arousal_labels in test_loader:
                valence_output, arousal_output = model(batch_features)
                
                # Compute separate losses for valence and arousal
                valence_loss = criterion(valence_output, batch_valence_labels)
                arousal_loss = criterion(arousal_output, batch_arousal_labels)
                
                # Accumulate separate losses
                valence_test_loss += valence_loss.item() * batch_features.size(0)
                arousal_test_loss += arousal_loss.item() * batch_features.size(0)
            
        # Average losses over the entire test dataset
        valence_test_loss /= len(test_loader.dataset)
        arousal_test_loss /= len(test_loader.dataset)
        
        # Print separate losses for valence and arousal in the test set
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Test Loss: {valence_test_loss:.4f}, Arousal Test Loss: {arousal_test_loss:.4f}")

In [45]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1550, Arousal Train Loss: 0.1523
Epoch 1/50, Valence Test Loss: 0.1153, Arousal Test Loss: 0.1030
Epoch 2/50, Valence Train Loss: 0.1331, Arousal Train Loss: 0.1368
Epoch 2/50, Valence Test Loss: 0.1119, Arousal Test Loss: 0.1019
Epoch 3/50, Valence Train Loss: 0.1203, Arousal Train Loss: 0.1296
Epoch 3/50, Valence Test Loss: 0.1356, Arousal Test Loss: 0.1134
Epoch 4/50, Valence Train Loss: 0.0990, Arousal Train Loss: 0.0998
Epoch 4/50, Valence Test Loss: 0.1171, Arousal Test Loss: 0.1087
Epoch 5/50, Valence Train Loss: 0.0931, Arousal Train Loss: 0.0892
Epoch 5/50, Valence Test Loss: 0.1174, Arousal Test Loss: 0.1023
Epoch 6/50, Valence Train Loss: 0.0911, Arousal Train Loss: 0.0838
Epoch 6/50, Valence Test Loss: 0.1097, Arousal Test Loss: 0.1012
Epoch 7/50, Valence Train Loss: 0.0893, Arousal Train Loss: 0.0794
Epoch 7/50, Valence Test Loss: 0.1092, Arousal Test Loss: 0.1011
Epoch 8/50, Valence Train Loss: 0.0871, Arousal Train Loss: 0.0755
Epoch 8/5

In [13]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_weights = nn.Linear(hidden_size, 1)

    def forward(self, encoder_outputs):
        # Calculate attention scores
        attention_scores = self.attention_weights(encoder_outputs)
        
        # Apply softmax to get attention weights
        attention_weights = F.softmax(attention_scores, dim=1)
        
        # Apply attention weights to encoder outputs
        context_vector = torch.bmm(attention_weights.permute(0, 2, 1), encoder_outputs)
        
        return context_vector

class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention = Attention(hidden_size)
        self.fc_valence = nn.Linear(hidden_size, output_size)
        self.fc_arousal = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        # Apply attention mechanism to LSTM outputs
        context_vector = self.attention(out)
        
        # Apply dropout
        context_vector = self.dropout(context_vector)
        
        # Pass the context vector through fully connected layers for valence and arousal predictions
        valence_output = self.fc_valence(context_vector.squeeze(1))
        arousal_output = self.fc_arousal(context_vector.squeeze(1))
        
        return valence_output, arousal_output


In [14]:
def LSTM_Attention_Model(X_train, X_test):

    input_size = 512  # Assuming each frame is represented by a feature vector of size 512
    hidden_size = 256
    num_layers = 4
    output_size = 750  # Number of frames
    dropout = 0.5
    
    # Convert NumPy arrays to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    X_test_tensor = torch.FloatTensor(X_test)
    y_train_valence_tensor = torch.FloatTensor(y_train_valence)
    y_test_valence_tensor = torch.FloatTensor(y_test_valence)
    y_train_arousal_tensor = torch.FloatTensor(y_train_arousal)
    y_test_arousal_tensor = torch.FloatTensor(y_test_arousal)
    
    # Create DataLoader for training and testing sets
    train_dataset = TensorDataset(X_train_tensor, y_train_valence_tensor, y_train_arousal_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_valence_tensor, y_test_arousal_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    model = LSTMWithAttention(input_size, hidden_size, num_layers, output_size, dropout=dropout)
    criterion = CCCLoss()  # Mean Squared Error loss
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        valence_train_loss = 0.0
        arousal_train_loss = 0.0
        
        for batch_features, batch_valence_labels, batch_arousal_labels in train_loader:
            optimizer.zero_grad()
            valence_output, arousal_output = model(batch_features)
            
            # Compute separate losses for valence and arousal
            valence_loss = criterion(valence_output, batch_valence_labels)
            arousal_loss = criterion(arousal_output, batch_arousal_labels)
            
            # Accumulate separate losses
            valence_train_loss += valence_loss.item() * batch_features.size(0)
            arousal_train_loss += arousal_loss.item() * batch_features.size(0)
            
            # Backpropagation and optimization
            loss = valence_loss + arousal_loss
            loss.backward()
            optimizer.step()
        
        # Average losses over the entire dataset
        valence_train_loss /= len(train_loader.dataset)
        arousal_train_loss /= len(train_loader.dataset)
        
        # Print separate losses for valence and arousal
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Train Loss: {valence_train_loss:.4f}, Arousal Train Loss: {arousal_train_loss:.4f}")
    
        # Evaluation loop
        model.eval()
        valence_test_loss = 0.0
        arousal_test_loss = 0.0
        
        with torch.no_grad():
            for batch_features, batch_valence_labels, batch_arousal_labels in test_loader:
                valence_output, arousal_output = model(batch_features)
                
                # Compute separate losses for valence and arousal
                valence_loss = criterion(valence_output, batch_valence_labels)
                arousal_loss = criterion(arousal_output, batch_arousal_labels)
                
                # Accumulate separate losses
                valence_test_loss += valence_loss.item() * batch_features.size(0)
                arousal_test_loss += arousal_loss.item() * batch_features.size(0)
            
        # Average losses over the entire test dataset
        valence_test_loss /= len(test_loader.dataset)
        arousal_test_loss /= len(test_loader.dataset)
        
        # Print separate losses for valence and arousal in the test set
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Test Loss: {valence_test_loss:.4f}, Arousal Test Loss: {arousal_test_loss:.4f}")

In [48]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9234, Arousal Train Loss: 0.9706
Epoch 1/50, Valence Test Loss: 0.8949, Arousal Test Loss: 0.8951
Epoch 2/50, Valence Train Loss: 0.8084, Arousal Train Loss: 0.8803
Epoch 2/50, Valence Test Loss: 0.8393, Arousal Test Loss: 0.8426
Epoch 3/50, Valence Train Loss: 0.7756, Arousal Train Loss: 0.8426
Epoch 3/50, Valence Test Loss: 0.8173, Arousal Test Loss: 0.8393
Epoch 4/50, Valence Train Loss: 0.7605, Arousal Train Loss: 0.8246
Epoch 4/50, Valence Test Loss: 0.8241, Arousal Test Loss: 0.8357
Epoch 5/50, Valence Train Loss: 0.7495, Arousal Train Loss: 0.8193
Epoch 5/50, Valence Test Loss: 0.8173, Arousal Test Loss: 0.8342
Epoch 6/50, Valence Train Loss: 0.7395, Arousal Train Loss: 0.8075
Epoch 6/50, Valence Test Loss: 0.8124, Arousal Test Loss: 0.8390
Epoch 7/50, Valence Train Loss: 0.7088, Arousal Train Loss: 0.7659
Epoch 7/50, Valence Test Loss: 0.8233, Arousal Test Loss: 0.8407
Epoch 8/50, Valence Train Loss: 0.6728, Arousal Train Loss: 0.7340
Epoch 8/5

In [15]:
class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(BiGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc_valence = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional
        self.fc_arousal = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # Multiply by 2 for bidirectional
        out, _ = self.gru(x, h0)
        
        # Concatenate hidden states from both directions
        out = torch.cat((out[:, -1, :self.hidden_size], out[:, 0, self.hidden_size:]), dim=1)
        
        # Apply dropout
        out = self.dropout(out)
        
        # Pass the concatenated hidden states through fully connected layers for valence and arousal predictions
        valence_output = self.fc_valence(out)
        arousal_output = self.fc_arousal(out)
        
        return valence_output, arousal_output

In [16]:
def bi_GRU_model(X_train, X_test):

    input_size = 512  # Assuming each frame is represented by a feature vector of size 512
    hidden_size = 256
    num_layers = 4
    output_size = 750  # Number of frames
    dropout = 0.5
    
    # Convert NumPy arrays to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    X_test_tensor = torch.FloatTensor(X_test)
    y_train_valence_tensor = torch.FloatTensor(y_train_valence)
    y_test_valence_tensor = torch.FloatTensor(y_test_valence)
    y_train_arousal_tensor = torch.FloatTensor(y_train_arousal)
    y_test_arousal_tensor = torch.FloatTensor(y_test_arousal)
    
    # Create DataLoader for training and testing sets
    train_dataset = TensorDataset(X_train_tensor, y_train_valence_tensor, y_train_arousal_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_valence_tensor, y_test_arousal_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)
    
    model = BiGRU(input_size, hidden_size, num_layers, output_size, dropout=dropout)
    criterion = nn.MSELoss()
    ccc_criterion = CCCLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        valence_train_loss = 0.0
        arousal_train_loss = 0.0
        
        for batch_features, batch_valence_labels, batch_arousal_labels in train_loader:
            optimizer.zero_grad()
            valence_output, arousal_output = model(batch_features)
            
            # Compute separate losses for valence and arousal
            valence_loss = criterion(valence_output, batch_valence_labels)
            arousal_loss = criterion(arousal_output, batch_arousal_labels)
            
            # Accumulate separate losses
            valence_train_loss += valence_loss.item() * batch_features.size(0)
            arousal_train_loss += arousal_loss.item() * batch_features.size(0)
            
            # Backpropagation and optimization
            loss = valence_loss + arousal_loss
            loss.backward()
            optimizer.step()
        
        # Average losses over the entire dataset
        valence_train_loss /= len(train_loader.dataset)
        arousal_train_loss /= len(train_loader.dataset)
        
        # Print separate losses for valence and arousal
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Train Loss: {valence_train_loss:.4f}, Arousal Train Loss: {arousal_train_loss:.4f}")
    
    # Evaluation loop
        model.eval()
        valence_test_loss = 0.0
        arousal_test_loss = 0.0
        ccc_valence = 0.0
        ccc_arousal = 0.0
        
        with torch.no_grad():
            for batch_features, batch_valence_labels, batch_arousal_labels in test_loader:
                valence_output, arousal_output = model(batch_features)   
                
                # Compute CCC for valence and arousal
                ccc_valence += ccc_criterion(valence_output.squeeze(), batch_valence_labels).item() * batch_features.size(0)
                ccc_arousal += ccc_criterion(arousal_output.squeeze(), batch_arousal_labels).item() * batch_features.size(0)
                
                # Accumulate separate losses
                valence_test_loss += valence_loss.item() * batch_features.size(0)
                arousal_test_loss += arousal_loss.item() * batch_features.size(0)
            
        # Average losses over the entire test dataset
        valence_test_loss /= len(test_loader.dataset)
        arousal_test_loss /= len(test_loader.dataset)
    
    
        ccc_valence /= len(test_loader.dataset)
        ccc_arousal /= len(test_loader.dataset)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Valence Test Loss : {valence_test_loss:.4f}, Arousal Test Loss: {arousal_test_loss:.4f}")
        #print(f"Valence Test Loss (RMSE): {rmse_valence:.4f}, Arousal Test Loss (RMSE): {rmse_arousal:.4f}")
        print(f"Epoch {epoch+1}/{num_epochs}, Valence CCC Loss : {ccc_valence:.4f}, Arousal CCC Loss: {ccc_arousal:.4f}")

In [50]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1708, Arousal Train Loss: 0.1671
Epoch 1/50, Valence Test Loss : 0.0988, Arousal Test Loss: 0.0747
Epoch 1/50, Valence CCC Loss : 0.9085, Arousal CCC Loss: 0.9176
Epoch 2/50, Valence Train Loss: 0.1419, Arousal Train Loss: 0.1482
Epoch 2/50, Valence Test Loss : 0.1202, Arousal Test Loss: 0.1346
Epoch 2/50, Valence CCC Loss : 0.8976, Arousal CCC Loss: 0.9007
Epoch 3/50, Valence Train Loss: 0.1095, Arousal Train Loss: 0.1156
Epoch 3/50, Valence Test Loss : 0.0593, Arousal Test Loss: 0.0923
Epoch 3/50, Valence CCC Loss : 0.8853, Arousal CCC Loss: 0.9026
Epoch 4/50, Valence Train Loss: 0.1155, Arousal Train Loss: 0.1118
Epoch 4/50, Valence Test Loss : 0.1297, Arousal Test Loss: 0.1483
Epoch 4/50, Valence CCC Loss : 0.9170, Arousal CCC Loss: 0.9305
Epoch 5/50, Valence Train Loss: 0.1145, Arousal Train Loss: 0.1007
Epoch 5/50, Valence Test Loss : 0.0920, Arousal Test Loss: 0.0531
Epoch 5/50, Valence CCC Loss : 0.8653, Arousal CCC Loss: 0.9010
Epoch 6/50, Val

In [15]:
def read_tensors_from_folder(folder_path):
    df = pd.DataFrame(columns=['Category','Tensor_File'])
    #labels = []
    filenames = []
    category = []
    
    
    # Iterate through each file
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if '_C1_' in file:
            category = 'C1'
        elif '_C2_' in file:
            category = 'C2'
        elif '_C3_' in file:
            category = 'C3'
        elif '_C4_' in file:
            category = 'C4'
        elif '_C5_' in file:
            category = 'C5'
        elif '_C6_' in file:
            category = 'C6'
        else:
            category = 'unknown'
          
        # Extract label from the file name
        #label = extract_label_from_filename(file)
        # Append information to DataFrame
        df.loc[len(df)] = [category,file_path]
        
    return df

In [16]:
max_length = 750
SEWA_path = "../SEWAv02"
folder_path = 'vggpooled/'
df = read_tensors_from_folder(folder_path)
print(df.head())

  Category                                      Tensor_File
0       C6  vggpooled/SVH_C6_S169_P338_VC1_000002_000855.pt
1       C2  vggpooled/SSL_C2_S036_P072_VC1_003807_004318.pt
2       C6  vggpooled/SAL_C6_S191_P381_VC1_004792_006827.pt
3       C5  vggpooled/SSL_C5_S157_P314_VC1_000601_001101.pt
4       C6  vggpooled/SVH_C6_S168_P336_VC1_000002_002001.pt


In [17]:
df_C1_video = df[df['Category']=='C1']
df_C2_video = df[df['Category']=='C2']
df_C3_video = df[df['Category']=='C3']
df_C4_video = df[df['Category']=='C4']
df_C5_video = df[df['Category']=='C5']
df_C6_video = df[df['Category']=='C6']

In [52]:
def get_data(df):
    max_length = 750
    feature_tensors = []
    label_tensors_valence = []
    label_tensors_arousal = []
    for file in df['Tensor_File']:
        #Extract tensors
        l = file.split('/')[-1]
        file_name = l[:-3]
        
        arr = torch.load(file)
        feature_tensors.append(arr)
        
        #Extract Valence and Arousal
        label_file_path = os.path.join(SEWA_path, file_name)
        #print(label_file_path)
        for label_file in os.listdir(label_file_path):
            #print(label_file)
            if label_file.endswith('_Arousal_V_Aligned.csv'):
                #print('Here')
                arousal_df = pd.read_csv(os.path.join(label_file_path, label_file))
                arousal_values = arousal_df['arousal'].values[:750]  # Take first 750 frames
                padded_arousal_values = np.pad(arousal_values, (0, max_length - len(arousal_values)), mode='constant')
                label_tensors_arousal.append(padded_arousal_values)
            elif label_file.endswith('_Valence_V_Aligned.csv'):
                #print('Here')
                valence_df = pd.read_csv(os.path.join(label_file_path, label_file))
                valence_values = valence_df['valence'].values[:750]  # Take first 750 frames
                padded_valence_values = np.pad(valence_values, (0, max_length - len(valence_values)), mode='constant')
                label_tensors_valence.append(padded_valence_values)
    return [feature_tensors, label_tensors_valence, label_tensors_arousal]

In [76]:
# def normalize(features):
#     padded_features = pad_sequences(tensors, maxlen=max_length, padding='post', dtype='float32')
#     normalized_feature_tensors = []
#     for video_features in padded_features:
#         # Compute mean and standard deviation along the first axis (frames)
#         mean = np.mean(video_features, axis=0, keepdims=True)
#         std = np.std(video_features, axis=0, keepdims=True)
#         # Apply normalization to each frame
#         normalized_video_features = (video_features - mean) / (std + 1e-8)  # Adding a small epsilon to avoid division by zero
#         normalized_feature_tensors.append(normalized_video_features)

#     # Convert the list of normalized feature tensors back to a numpy array
#     normalized_feature_tensors = np.array(normalized_feature_tensors)

#     return normalized_feature_tensors

In [65]:
tensors, valence, arousal = get_data(df_C1_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (60, 750, 512)
X_test shape: (15, 750, 512)
y_train_valence shape: (60, 750)
y_test_valence shape: (15, 750)
y_train_arousal shape: (60, 750)
y_test_arousal shape: (15, 750)


In [58]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1412, Arousal Train Loss: 0.1087
Epoch 1/50, Valence Test Loss: 0.1892, Arousal Test Loss: 0.1361
Epoch 2/50, Valence Train Loss: 0.1251, Arousal Train Loss: 0.0960
Epoch 2/50, Valence Test Loss: 0.1696, Arousal Test Loss: 0.1209
Epoch 3/50, Valence Train Loss: 0.1063, Arousal Train Loss: 0.0823
Epoch 3/50, Valence Test Loss: 0.1397, Arousal Test Loss: 0.1007
Epoch 4/50, Valence Train Loss: 0.0835, Arousal Train Loss: 0.0674
Epoch 4/50, Valence Test Loss: 0.1187, Arousal Test Loss: 0.0848
Epoch 5/50, Valence Train Loss: 0.0667, Arousal Train Loss: 0.0578
Epoch 5/50, Valence Test Loss: 0.0982, Arousal Test Loss: 0.0712
Epoch 6/50, Valence Train Loss: 0.0505, Arousal Train Loss: 0.0443
Epoch 6/50, Valence Test Loss: 0.0939, Arousal Test Loss: 0.0700
Epoch 7/50, Valence Train Loss: 0.0439, Arousal Train Loss: 0.0363
Epoch 7/50, Valence Test Loss: 0.0911, Arousal Test Loss: 0.0691
Epoch 8/50, Valence Train Loss: 0.0426, Arousal Train Loss: 0.0332
Epoch 8/5

In [60]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1290, Arousal Train Loss: 0.0972
Epoch 1/50, Valence Test Loss: 0.1783, Arousal Test Loss: 0.1253
Epoch 2/50, Valence Train Loss: 0.1159, Arousal Train Loss: 0.0867
Epoch 2/50, Valence Test Loss: 0.1394, Arousal Test Loss: 0.0982
Epoch 3/50, Valence Train Loss: 0.0902, Arousal Train Loss: 0.0682
Epoch 3/50, Valence Test Loss: 0.0845, Arousal Test Loss: 0.0616
Epoch 4/50, Valence Train Loss: 0.0732, Arousal Train Loss: 0.0597
Epoch 4/50, Valence Test Loss: 0.0904, Arousal Test Loss: 0.0592
Epoch 5/50, Valence Train Loss: 0.0607, Arousal Train Loss: 0.0485
Epoch 5/50, Valence Test Loss: 0.0786, Arousal Test Loss: 0.0520
Epoch 6/50, Valence Train Loss: 0.0516, Arousal Train Loss: 0.0435
Epoch 6/50, Valence Test Loss: 0.0622, Arousal Test Loss: 0.0430
Epoch 7/50, Valence Train Loss: 0.0450, Arousal Train Loss: 0.0394
Epoch 7/50, Valence Test Loss: 0.0639, Arousal Test Loss: 0.0450
Epoch 8/50, Valence Train Loss: 0.0398, Arousal Train Loss: 0.0333
Epoch 8/5

In [28]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9838, Arousal Train Loss: 0.9785
Epoch 1/50, Valence Test Loss: 0.9210, Arousal Test Loss: 0.8862
Epoch 2/50, Valence Train Loss: 0.8838, Arousal Train Loss: 0.9053
Epoch 2/50, Valence Test Loss: 0.7364, Arousal Test Loss: 0.6856
Epoch 3/50, Valence Train Loss: 0.7717, Arousal Train Loss: 0.7997
Epoch 3/50, Valence Test Loss: 0.6190, Arousal Test Loss: 0.5812
Epoch 4/50, Valence Train Loss: 0.7324, Arousal Train Loss: 0.7585
Epoch 4/50, Valence Test Loss: 0.5583, Arousal Test Loss: 0.4978
Epoch 5/50, Valence Train Loss: 0.6646, Arousal Train Loss: 0.7039
Epoch 5/50, Valence Test Loss: 0.5458, Arousal Test Loss: 0.4559
Epoch 6/50, Valence Train Loss: 0.6438, Arousal Train Loss: 0.6761
Epoch 6/50, Valence Test Loss: 0.5065, Arousal Test Loss: 0.4174
Epoch 7/50, Valence Train Loss: 0.6132, Arousal Train Loss: 0.6475
Epoch 7/50, Valence Test Loss: 0.4558, Arousal Test Loss: 0.3919
Epoch 8/50, Valence Train Loss: 0.5920, Arousal Train Loss: 0.6287
Epoch 8/5

In [29]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1426, Arousal Train Loss: 0.1123
Epoch 1/50, Valence Test Loss : 0.1374, Arousal Test Loss: 0.1003
Epoch 1/50, Valence CCC Loss : 0.9403, Arousal CCC Loss: 0.9176
Epoch 2/50, Valence Train Loss: 0.1175, Arousal Train Loss: 0.0929
Epoch 2/50, Valence Test Loss : 0.1023, Arousal Test Loss: 0.0793
Epoch 2/50, Valence CCC Loss : 0.8018, Arousal CCC Loss: 0.7642
Epoch 3/50, Valence Train Loss: 0.0810, Arousal Train Loss: 0.0662
Epoch 3/50, Valence Test Loss : 0.0771, Arousal Test Loss: 0.0579
Epoch 3/50, Valence CCC Loss : 0.6675, Arousal CCC Loss: 0.6188
Epoch 4/50, Valence Train Loss: 0.0656, Arousal Train Loss: 0.0638
Epoch 4/50, Valence Test Loss : 0.0639, Arousal Test Loss: 0.0648
Epoch 4/50, Valence CCC Loss : 0.6096, Arousal CCC Loss: 0.5594
Epoch 5/50, Valence Train Loss: 0.0527, Arousal Train Loss: 0.0477
Epoch 5/50, Valence Test Loss : 0.0518, Arousal Test Loss: 0.0488
Epoch 5/50, Valence CCC Loss : 0.6591, Arousal CCC Loss: 0.6246
Epoch 6/50, Val

In [30]:
tensors, valence, arousal = get_data(df_C2_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (64, 750, 512)
X_test shape: (17, 750, 512)
y_train_valence shape: (64, 750)
y_test_valence shape: (17, 750)
y_train_arousal shape: (64, 750)
y_test_arousal shape: (17, 750)


In [31]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.3091, Arousal Train Loss: 0.3524
Epoch 1/50, Valence Test Loss: 0.2706, Arousal Test Loss: 0.3020
Epoch 2/50, Valence Train Loss: 0.2858, Arousal Train Loss: 0.3241
Epoch 2/50, Valence Test Loss: 0.2600, Arousal Test Loss: 0.2834
Epoch 3/50, Valence Train Loss: 0.2561, Arousal Train Loss: 0.2858
Epoch 3/50, Valence Test Loss: 0.2546, Arousal Test Loss: 0.2625
Epoch 4/50, Valence Train Loss: 0.2227, Arousal Train Loss: 0.2351
Epoch 4/50, Valence Test Loss: 0.2566, Arousal Test Loss: 0.2420
Epoch 5/50, Valence Train Loss: 0.1871, Arousal Train Loss: 0.1854
Epoch 5/50, Valence Test Loss: 0.2417, Arousal Test Loss: 0.2175
Epoch 6/50, Valence Train Loss: 0.1579, Arousal Train Loss: 0.1520
Epoch 6/50, Valence Test Loss: 0.2481, Arousal Test Loss: 0.2166
Epoch 7/50, Valence Train Loss: 0.1405, Arousal Train Loss: 0.1270
Epoch 7/50, Valence Test Loss: 0.2869, Arousal Test Loss: 0.2582
Epoch 8/50, Valence Train Loss: 0.1262, Arousal Train Loss: 0.1212
Epoch 8/5

In [32]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.2963, Arousal Train Loss: 0.3376
Epoch 1/50, Valence Test Loss: 0.2605, Arousal Test Loss: 0.2915
Epoch 2/50, Valence Train Loss: 0.2745, Arousal Train Loss: 0.3124
Epoch 2/50, Valence Test Loss: 0.2356, Arousal Test Loss: 0.2538
Epoch 3/50, Valence Train Loss: 0.2270, Arousal Train Loss: 0.2442
Epoch 3/50, Valence Test Loss: 0.2813, Arousal Test Loss: 0.2490
Epoch 4/50, Valence Train Loss: 0.1975, Arousal Train Loss: 0.1829
Epoch 4/50, Valence Test Loss: 0.2276, Arousal Test Loss: 0.1960
Epoch 5/50, Valence Train Loss: 0.1589, Arousal Train Loss: 0.1464
Epoch 5/50, Valence Test Loss: 0.2134, Arousal Test Loss: 0.1932
Epoch 6/50, Valence Train Loss: 0.1479, Arousal Train Loss: 0.1389
Epoch 6/50, Valence Test Loss: 0.2213, Arousal Test Loss: 0.2037
Epoch 7/50, Valence Train Loss: 0.1447, Arousal Train Loss: 0.1359
Epoch 7/50, Valence Test Loss: 0.2309, Arousal Test Loss: 0.2165
Epoch 8/50, Valence Train Loss: 0.1423, Arousal Train Loss: 0.1335
Epoch 8/5

In [33]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9946, Arousal Train Loss: 1.0018
Epoch 1/50, Valence Test Loss: 0.9972, Arousal Test Loss: 1.0175
Epoch 2/50, Valence Train Loss: 0.9580, Arousal Train Loss: 0.9573
Epoch 2/50, Valence Test Loss: 0.9902, Arousal Test Loss: 0.9427
Epoch 3/50, Valence Train Loss: 0.8901, Arousal Train Loss: 0.8595
Epoch 3/50, Valence Test Loss: 0.9544, Arousal Test Loss: 0.8611
Epoch 4/50, Valence Train Loss: 0.8583, Arousal Train Loss: 0.8290
Epoch 4/50, Valence Test Loss: 0.9283, Arousal Test Loss: 0.8449
Epoch 5/50, Valence Train Loss: 0.8249, Arousal Train Loss: 0.8019
Epoch 5/50, Valence Test Loss: 0.9048, Arousal Test Loss: 0.8248
Epoch 6/50, Valence Train Loss: 0.7867, Arousal Train Loss: 0.7731
Epoch 6/50, Valence Test Loss: 0.9017, Arousal Test Loss: 0.8255
Epoch 7/50, Valence Train Loss: 0.7640, Arousal Train Loss: 0.7688
Epoch 7/50, Valence Test Loss: 0.8863, Arousal Test Loss: 0.8193
Epoch 8/50, Valence Train Loss: 0.7429, Arousal Train Loss: 0.7584
Epoch 8/5

In [34]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.3159, Arousal Train Loss: 0.3562
Epoch 1/50, Valence Test Loss : 0.2801, Arousal Test Loss: 0.3062
Epoch 1/50, Valence CCC Loss : 0.9926, Arousal CCC Loss: 0.9701
Epoch 2/50, Valence Train Loss: 0.2635, Arousal Train Loss: 0.2888
Epoch 2/50, Valence Test Loss : 0.2641, Arousal Test Loss: 0.2973
Epoch 2/50, Valence CCC Loss : 0.9444, Arousal CCC Loss: 0.9084
Epoch 3/50, Valence Train Loss: 0.2177, Arousal Train Loss: 0.2193
Epoch 3/50, Valence Test Loss : 0.2374, Arousal Test Loss: 0.2124
Epoch 3/50, Valence CCC Loss : 0.9464, Arousal CCC Loss: 0.8866
Epoch 4/50, Valence Train Loss: 0.1860, Arousal Train Loss: 0.1724
Epoch 4/50, Valence Test Loss : 0.2061, Arousal Test Loss: 0.2031
Epoch 4/50, Valence CCC Loss : 0.9132, Arousal CCC Loss: 0.8503
Epoch 5/50, Valence Train Loss: 0.1703, Arousal Train Loss: 0.1492
Epoch 5/50, Valence Test Loss : 0.1799, Arousal Test Loss: 0.1402
Epoch 5/50, Valence CCC Loss : 0.9505, Arousal CCC Loss: 0.8725
Epoch 6/50, Val

In [7]:
tensors, valence, arousal = get_data(df_C3_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (57, 750, 512)
X_test shape: (15, 750, 512)
y_train_valence shape: (57, 750)
y_test_valence shape: (15, 750)
y_train_arousal shape: (57, 750)
y_test_arousal shape: (15, 750)


In [44]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1363, Arousal Train Loss: 0.0894
Epoch 1/50, Valence Test Loss: 0.1056, Arousal Test Loss: 0.0417
Epoch 2/50, Valence Train Loss: 0.1239, Arousal Train Loss: 0.0810
Epoch 2/50, Valence Test Loss: 0.0995, Arousal Test Loss: 0.0386
Epoch 3/50, Valence Train Loss: 0.1155, Arousal Train Loss: 0.0755
Epoch 3/50, Valence Test Loss: 0.0956, Arousal Test Loss: 0.0373
Epoch 4/50, Valence Train Loss: 0.1095, Arousal Train Loss: 0.0725
Epoch 4/50, Valence Test Loss: 0.0926, Arousal Test Loss: 0.0368
Epoch 5/50, Valence Train Loss: 0.1025, Arousal Train Loss: 0.0689
Epoch 5/50, Valence Test Loss: 0.0898, Arousal Test Loss: 0.0365
Epoch 6/50, Valence Train Loss: 0.0945, Arousal Train Loss: 0.0653
Epoch 6/50, Valence Test Loss: 0.0886, Arousal Test Loss: 0.0374
Epoch 7/50, Valence Train Loss: 0.0851, Arousal Train Loss: 0.0621
Epoch 7/50, Valence Test Loss: 0.0882, Arousal Test Loss: 0.0379
Epoch 8/50, Valence Train Loss: 0.0772, Arousal Train Loss: 0.0581
Epoch 8/5

In [45]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1191, Arousal Train Loss: 0.0726
Epoch 1/50, Valence Test Loss: 0.0929, Arousal Test Loss: 0.0300
Epoch 2/50, Valence Train Loss: 0.1124, Arousal Train Loss: 0.0714
Epoch 2/50, Valence Test Loss: 0.0876, Arousal Test Loss: 0.0325
Epoch 3/50, Valence Train Loss: 0.1031, Arousal Train Loss: 0.0721
Epoch 3/50, Valence Test Loss: 0.0821, Arousal Test Loss: 0.0303
Epoch 4/50, Valence Train Loss: 0.0974, Arousal Train Loss: 0.0683
Epoch 4/50, Valence Test Loss: 0.0816, Arousal Test Loss: 0.0312
Epoch 5/50, Valence Train Loss: 0.0951, Arousal Train Loss: 0.0698
Epoch 5/50, Valence Test Loss: 0.0846, Arousal Test Loss: 0.0318
Epoch 6/50, Valence Train Loss: 0.0826, Arousal Train Loss: 0.0657
Epoch 6/50, Valence Test Loss: 0.0811, Arousal Test Loss: 0.0292
Epoch 7/50, Valence Train Loss: 0.0814, Arousal Train Loss: 0.0656
Epoch 7/50, Valence Test Loss: 0.0893, Arousal Test Loss: 0.0306
Epoch 8/50, Valence Train Loss: 0.0766, Arousal Train Loss: 0.0678
Epoch 8/5

In [46]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9988, Arousal Train Loss: 1.0056
Epoch 1/50, Valence Test Loss: 0.9890, Arousal Test Loss: 1.0066
Epoch 2/50, Valence Train Loss: 0.9770, Arousal Train Loss: 0.9827
Epoch 2/50, Valence Test Loss: 0.9736, Arousal Test Loss: 0.9973
Epoch 3/50, Valence Train Loss: 0.9509, Arousal Train Loss: 0.9760
Epoch 3/50, Valence Test Loss: 0.9507, Arousal Test Loss: 1.0018
Epoch 4/50, Valence Train Loss: 0.9083, Arousal Train Loss: 0.9485
Epoch 4/50, Valence Test Loss: 0.9852, Arousal Test Loss: 0.9780
Epoch 5/50, Valence Train Loss: 0.8914, Arousal Train Loss: 0.9271
Epoch 5/50, Valence Test Loss: 1.0095, Arousal Test Loss: 0.9709
Epoch 6/50, Valence Train Loss: 0.8768, Arousal Train Loss: 0.9166
Epoch 6/50, Valence Test Loss: 1.0192, Arousal Test Loss: 0.9484
Epoch 7/50, Valence Train Loss: 0.8468, Arousal Train Loss: 0.8883
Epoch 7/50, Valence Test Loss: 1.0372, Arousal Test Loss: 0.9456
Epoch 8/50, Valence Train Loss: 0.8383, Arousal Train Loss: 0.8746
Epoch 8/5

In [17]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1426, Arousal Train Loss: 0.0816
Epoch 1/50, Valence Test Loss : 0.1437, Arousal Test Loss: 0.0773
Epoch 1/50, Valence CCC Loss : 0.9941, Arousal CCC Loss: 1.0196
Epoch 2/50, Valence Train Loss: 0.1317, Arousal Train Loss: 0.0767
Epoch 2/50, Valence Test Loss : 0.1367, Arousal Test Loss: 0.0787
Epoch 2/50, Valence CCC Loss : 0.9898, Arousal CCC Loss: 0.9763
Epoch 3/50, Valence Train Loss: 0.1168, Arousal Train Loss: 0.0734
Epoch 3/50, Valence Test Loss : 0.1002, Arousal Test Loss: 0.0612
Epoch 3/50, Valence CCC Loss : 0.9888, Arousal CCC Loss: 0.9604
Epoch 4/50, Valence Train Loss: 0.1100, Arousal Train Loss: 0.0711
Epoch 4/50, Valence Test Loss : 0.1022, Arousal Test Loss: 0.0754
Epoch 4/50, Valence CCC Loss : 0.9843, Arousal CCC Loss: 0.9674
Epoch 5/50, Valence Train Loss: 0.0959, Arousal Train Loss: 0.0661
Epoch 5/50, Valence Test Loss : 0.1054, Arousal Test Loss: 0.0758
Epoch 5/50, Valence CCC Loss : 0.9664, Arousal CCC Loss: 0.9557
Epoch 6/50, Val

In [42]:
tensors, valence, arousal = get_data(df_C4_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (47, 750, 512)
X_test shape: (12, 750, 512)
y_train_valence shape: (47, 750)
y_test_valence shape: (12, 750)
y_train_arousal shape: (47, 750)
y_test_arousal shape: (12, 750)


In [19]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.2130, Arousal Train Loss: 0.1910
Epoch 1/50, Valence Test Loss: 0.1814, Arousal Test Loss: 0.2453
Epoch 2/50, Valence Train Loss: 0.1997, Arousal Train Loss: 0.1806
Epoch 2/50, Valence Test Loss: 0.1716, Arousal Test Loss: 0.2393
Epoch 3/50, Valence Train Loss: 0.1829, Arousal Train Loss: 0.1717
Epoch 3/50, Valence Test Loss: 0.1660, Arousal Test Loss: 0.2380
Epoch 4/50, Valence Train Loss: 0.1657, Arousal Train Loss: 0.1626
Epoch 4/50, Valence Test Loss: 0.1622, Arousal Test Loss: 0.2371
Epoch 5/50, Valence Train Loss: 0.1485, Arousal Train Loss: 0.1511
Epoch 5/50, Valence Test Loss: 0.1634, Arousal Test Loss: 0.2397
Epoch 6/50, Valence Train Loss: 0.1275, Arousal Train Loss: 0.1375
Epoch 6/50, Valence Test Loss: 0.1665, Arousal Test Loss: 0.2424
Epoch 7/50, Valence Train Loss: 0.1062, Arousal Train Loss: 0.1210
Epoch 7/50, Valence Test Loss: 0.1572, Arousal Test Loss: 0.2318
Epoch 8/50, Valence Train Loss: 0.1082, Arousal Train Loss: 0.1082
Epoch 8/5

In [20]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1966, Arousal Train Loss: 0.1743
Epoch 1/50, Valence Test Loss: 0.1664, Arousal Test Loss: 0.2334
Epoch 2/50, Valence Train Loss: 0.1845, Arousal Train Loss: 0.1704
Epoch 2/50, Valence Test Loss: 0.1611, Arousal Test Loss: 0.2406
Epoch 3/50, Valence Train Loss: 0.1651, Arousal Train Loss: 0.1721
Epoch 3/50, Valence Test Loss: 0.1550, Arousal Test Loss: 0.2369
Epoch 4/50, Valence Train Loss: 0.1477, Arousal Train Loss: 0.1631
Epoch 4/50, Valence Test Loss: 0.1594, Arousal Test Loss: 0.2411
Epoch 5/50, Valence Train Loss: 0.1318, Arousal Train Loss: 0.1577
Epoch 5/50, Valence Test Loss: 0.1606, Arousal Test Loss: 0.2413
Epoch 6/50, Valence Train Loss: 0.1151, Arousal Train Loss: 0.1469
Epoch 6/50, Valence Test Loss: 0.1726, Arousal Test Loss: 0.2511
Epoch 7/50, Valence Train Loss: 0.1093, Arousal Train Loss: 0.1390
Epoch 7/50, Valence Test Loss: 0.1499, Arousal Test Loss: 0.2397
Epoch 8/50, Valence Train Loss: 0.0945, Arousal Train Loss: 0.1202
Epoch 8/5

In [21]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9963, Arousal Train Loss: 0.9961
Epoch 1/50, Valence Test Loss: 0.9830, Arousal Test Loss: 0.9807
Epoch 2/50, Valence Train Loss: 0.9560, Arousal Train Loss: 0.9764
Epoch 2/50, Valence Test Loss: 0.9218, Arousal Test Loss: 0.9317
Epoch 3/50, Valence Train Loss: 0.8642, Arousal Train Loss: 0.9144
Epoch 3/50, Valence Test Loss: 0.8177, Arousal Test Loss: 0.8740
Epoch 4/50, Valence Train Loss: 0.8031, Arousal Train Loss: 0.8616
Epoch 4/50, Valence Test Loss: 0.7937, Arousal Test Loss: 0.8499
Epoch 5/50, Valence Train Loss: 0.7792, Arousal Train Loss: 0.8424
Epoch 5/50, Valence Test Loss: 0.7743, Arousal Test Loss: 0.8286
Epoch 6/50, Valence Train Loss: 0.7493, Arousal Train Loss: 0.8177
Epoch 6/50, Valence Test Loss: 0.7833, Arousal Test Loss: 0.8172
Epoch 7/50, Valence Train Loss: 0.7344, Arousal Train Loss: 0.8099
Epoch 7/50, Valence Test Loss: 0.7852, Arousal Test Loss: 0.8169
Epoch 8/50, Valence Train Loss: 0.7214, Arousal Train Loss: 0.7991
Epoch 8/5

In [22]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.2136, Arousal Train Loss: 0.1957
Epoch 1/50, Valence Test Loss : 0.2240, Arousal Test Loss: 0.2098
Epoch 1/50, Valence CCC Loss : 0.9849, Arousal CCC Loss: 0.9869
Epoch 2/50, Valence Train Loss: 0.1936, Arousal Train Loss: 0.1875
Epoch 2/50, Valence Test Loss : 0.1715, Arousal Test Loss: 0.1752
Epoch 2/50, Valence CCC Loss : 0.9660, Arousal CCC Loss: 0.9621
Epoch 3/50, Valence Train Loss: 0.1694, Arousal Train Loss: 0.1785
Epoch 3/50, Valence Test Loss : 0.1775, Arousal Test Loss: 0.1705
Epoch 3/50, Valence CCC Loss : 0.9287, Arousal CCC Loss: 0.9282
Epoch 4/50, Valence Train Loss: 0.1403, Arousal Train Loss: 0.1624
Epoch 4/50, Valence Test Loss : 0.1366, Arousal Test Loss: 0.1307
Epoch 4/50, Valence CCC Loss : 0.9102, Arousal CCC Loss: 0.9052
Epoch 5/50, Valence Train Loss: 0.1130, Arousal Train Loss: 0.1456
Epoch 5/50, Valence Test Loss : 0.1179, Arousal Test Loss: 0.1225
Epoch 5/50, Valence CCC Loss : 0.9338, Arousal CCC Loss: 0.9204
Epoch 6/50, Val

In [23]:
tensors, valence, arousal = get_data(df_C5_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (48, 750, 512)
X_test shape: (12, 750, 512)
y_train_valence shape: (48, 750)
y_test_valence shape: (12, 750)
y_train_arousal shape: (48, 750)
y_test_arousal shape: (12, 750)


In [24]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1315, Arousal Train Loss: 0.0851
Epoch 1/50, Valence Test Loss: 0.1582, Arousal Test Loss: 0.1049
Epoch 2/50, Valence Train Loss: 0.1237, Arousal Train Loss: 0.0785
Epoch 2/50, Valence Test Loss: 0.1534, Arousal Test Loss: 0.0997
Epoch 3/50, Valence Train Loss: 0.1181, Arousal Train Loss: 0.0735
Epoch 3/50, Valence Test Loss: 0.1526, Arousal Test Loss: 0.0960
Epoch 4/50, Valence Train Loss: 0.1162, Arousal Train Loss: 0.0701
Epoch 4/50, Valence Test Loss: 0.1516, Arousal Test Loss: 0.0940
Epoch 5/50, Valence Train Loss: 0.1145, Arousal Train Loss: 0.0683
Epoch 5/50, Valence Test Loss: 0.1531, Arousal Test Loss: 0.0927
Epoch 6/50, Valence Train Loss: 0.1138, Arousal Train Loss: 0.0665
Epoch 6/50, Valence Test Loss: 0.1545, Arousal Test Loss: 0.0918
Epoch 7/50, Valence Train Loss: 0.1131, Arousal Train Loss: 0.0653
Epoch 7/50, Valence Test Loss: 0.1543, Arousal Test Loss: 0.0906
Epoch 8/50, Valence Train Loss: 0.1119, Arousal Train Loss: 0.0640
Epoch 8/5

In [25]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1144, Arousal Train Loss: 0.0696
Epoch 1/50, Valence Test Loss: 0.1487, Arousal Test Loss: 0.0926
Epoch 2/50, Valence Train Loss: 0.1132, Arousal Train Loss: 0.0673
Epoch 2/50, Valence Test Loss: 0.1510, Arousal Test Loss: 0.0903
Epoch 3/50, Valence Train Loss: 0.1130, Arousal Train Loss: 0.0662
Epoch 3/50, Valence Test Loss: 0.1507, Arousal Test Loss: 0.0890
Epoch 4/50, Valence Train Loss: 0.1116, Arousal Train Loss: 0.0642
Epoch 4/50, Valence Test Loss: 0.1504, Arousal Test Loss: 0.0893
Epoch 5/50, Valence Train Loss: 0.1113, Arousal Train Loss: 0.0640
Epoch 5/50, Valence Test Loss: 0.1514, Arousal Test Loss: 0.0887
Epoch 6/50, Valence Train Loss: 0.1112, Arousal Train Loss: 0.0629
Epoch 6/50, Valence Test Loss: 0.1543, Arousal Test Loss: 0.0885
Epoch 7/50, Valence Train Loss: 0.1119, Arousal Train Loss: 0.0626
Epoch 7/50, Valence Test Loss: 0.1543, Arousal Test Loss: 0.0881
Epoch 8/50, Valence Train Loss: 0.1102, Arousal Train Loss: 0.0616
Epoch 8/5

In [26]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9929, Arousal Train Loss: 1.0032
Epoch 1/50, Valence Test Loss: 0.9750, Arousal Test Loss: 0.9892
Epoch 2/50, Valence Train Loss: 0.9642, Arousal Train Loss: 0.9802
Epoch 2/50, Valence Test Loss: 0.8871, Arousal Test Loss: 0.9635
Epoch 3/50, Valence Train Loss: 0.9332, Arousal Train Loss: 0.9626
Epoch 3/50, Valence Test Loss: 0.8394, Arousal Test Loss: 0.9350
Epoch 4/50, Valence Train Loss: 0.9070, Arousal Train Loss: 0.9314
Epoch 4/50, Valence Test Loss: 0.8450, Arousal Test Loss: 0.9299
Epoch 5/50, Valence Train Loss: 0.8891, Arousal Train Loss: 0.8990
Epoch 5/50, Valence Test Loss: 0.8494, Arousal Test Loss: 0.9134
Epoch 6/50, Valence Train Loss: 0.8728, Arousal Train Loss: 0.8734
Epoch 6/50, Valence Test Loss: 0.8648, Arousal Test Loss: 0.8937
Epoch 7/50, Valence Train Loss: 0.8783, Arousal Train Loss: 0.8564
Epoch 7/50, Valence Test Loss: 0.8849, Arousal Test Loss: 0.8728
Epoch 8/50, Valence Train Loss: 0.8678, Arousal Train Loss: 0.8361
Epoch 8/5

In [27]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1437, Arousal Train Loss: 0.0992
Epoch 1/50, Valence Test Loss : 0.1676, Arousal Test Loss: 0.1183
Epoch 1/50, Valence CCC Loss : 0.9934, Arousal CCC Loss: 0.9971
Epoch 2/50, Valence Train Loss: 0.1391, Arousal Train Loss: 0.0936
Epoch 2/50, Valence Test Loss : 0.1596, Arousal Test Loss: 0.1063
Epoch 2/50, Valence CCC Loss : 1.0469, Arousal CCC Loss: 0.9990
Epoch 3/50, Valence Train Loss: 0.1311, Arousal Train Loss: 0.0842
Epoch 3/50, Valence Test Loss : 0.1508, Arousal Test Loss: 0.0838
Epoch 3/50, Valence CCC Loss : 1.0276, Arousal CCC Loss: 1.0085
Epoch 4/50, Valence Train Loss: 0.1282, Arousal Train Loss: 0.0814
Epoch 4/50, Valence Test Loss : 0.1182, Arousal Test Loss: 0.0675
Epoch 4/50, Valence CCC Loss : 1.0259, Arousal CCC Loss: 1.0097
Epoch 5/50, Valence Train Loss: 0.1269, Arousal Train Loss: 0.0804
Epoch 5/50, Valence Test Loss : 0.1509, Arousal Test Loss: 0.1004
Epoch 5/50, Valence CCC Loss : 1.0687, Arousal CCC Loss: 1.0040
Epoch 6/50, Val

In [28]:
tensors, valence, arousal = get_data(df_C6_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (55, 750, 512)
X_test shape: (14, 750, 512)
y_train_valence shape: (55, 750)
y_test_valence shape: (14, 750)
y_train_arousal shape: (55, 750)
y_test_arousal shape: (14, 750)


In [29]:
RNN_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1462, Arousal Train Loss: 0.1391
Epoch 1/50, Valence Test Loss: 0.1780, Arousal Test Loss: 0.1729
Epoch 2/50, Valence Train Loss: 0.1324, Arousal Train Loss: 0.1281
Epoch 2/50, Valence Test Loss: 0.1726, Arousal Test Loss: 0.1733
Epoch 3/50, Valence Train Loss: 0.1228, Arousal Train Loss: 0.1216
Epoch 3/50, Valence Test Loss: 0.1677, Arousal Test Loss: 0.1764
Epoch 4/50, Valence Train Loss: 0.1082, Arousal Train Loss: 0.1133
Epoch 4/50, Valence Test Loss: 0.1651, Arousal Test Loss: 0.1815
Epoch 5/50, Valence Train Loss: 0.0909, Arousal Train Loss: 0.1023
Epoch 5/50, Valence Test Loss: 0.1689, Arousal Test Loss: 0.1935
Epoch 6/50, Valence Train Loss: 0.0762, Arousal Train Loss: 0.0888
Epoch 6/50, Valence Test Loss: 0.1762, Arousal Test Loss: 0.2092
Epoch 7/50, Valence Train Loss: 0.0641, Arousal Train Loss: 0.0740
Epoch 7/50, Valence Test Loss: 0.1671, Arousal Test Loss: 0.2118
Epoch 8/50, Valence Train Loss: 0.0585, Arousal Train Loss: 0.0655
Epoch 8/5

In [30]:
LSTM_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1313, Arousal Train Loss: 0.1240
Epoch 1/50, Valence Test Loss: 0.1655, Arousal Test Loss: 0.1622
Epoch 2/50, Valence Train Loss: 0.1227, Arousal Train Loss: 0.1228
Epoch 2/50, Valence Test Loss: 0.1600, Arousal Test Loss: 0.1699
Epoch 3/50, Valence Train Loss: 0.1016, Arousal Train Loss: 0.1225
Epoch 3/50, Valence Test Loss: 0.1581, Arousal Test Loss: 0.1802
Epoch 4/50, Valence Train Loss: 0.0851, Arousal Train Loss: 0.1218
Epoch 4/50, Valence Test Loss: 0.1443, Arousal Test Loss: 0.1735
Epoch 5/50, Valence Train Loss: 0.0629, Arousal Train Loss: 0.1080
Epoch 5/50, Valence Test Loss: 0.1486, Arousal Test Loss: 0.1815
Epoch 6/50, Valence Train Loss: 0.0535, Arousal Train Loss: 0.0978
Epoch 6/50, Valence Test Loss: 0.1889, Arousal Test Loss: 0.2095
Epoch 7/50, Valence Train Loss: 0.0643, Arousal Train Loss: 0.0830
Epoch 7/50, Valence Test Loss: 0.1663, Arousal Test Loss: 0.2040
Epoch 8/50, Valence Train Loss: 0.0582, Arousal Train Loss: 0.0807
Epoch 8/5

In [31]:
LSTM_Attention_Model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.9991, Arousal Train Loss: 1.0002
Epoch 1/50, Valence Test Loss: 0.9572, Arousal Test Loss: 0.9852
Epoch 2/50, Valence Train Loss: 0.9763, Arousal Train Loss: 0.9587
Epoch 2/50, Valence Test Loss: 0.8872, Arousal Test Loss: 0.9644
Epoch 3/50, Valence Train Loss: 0.9266, Arousal Train Loss: 0.9023
Epoch 3/50, Valence Test Loss: 0.8743, Arousal Test Loss: 0.8947
Epoch 4/50, Valence Train Loss: 0.8936, Arousal Train Loss: 0.8855
Epoch 4/50, Valence Test Loss: 0.8258, Arousal Test Loss: 0.8814
Epoch 5/50, Valence Train Loss: 0.8642, Arousal Train Loss: 0.8635
Epoch 5/50, Valence Test Loss: 0.7749, Arousal Test Loss: 0.8851
Epoch 6/50, Valence Train Loss: 0.8438, Arousal Train Loss: 0.8471
Epoch 6/50, Valence Test Loss: 0.7611, Arousal Test Loss: 0.8802
Epoch 7/50, Valence Train Loss: 0.8202, Arousal Train Loss: 0.8476
Epoch 7/50, Valence Test Loss: 0.7571, Arousal Test Loss: 0.8761
Epoch 8/50, Valence Train Loss: 0.8044, Arousal Train Loss: 0.8476
Epoch 8/5

In [32]:
bi_GRU_model(X_train, X_test)

Epoch 1/50, Valence Train Loss: 0.1480, Arousal Train Loss: 0.1400
Epoch 1/50, Valence Test Loss : 0.1633, Arousal Test Loss: 0.1457
Epoch 1/50, Valence CCC Loss : 0.9926, Arousal CCC Loss: 1.0001
Epoch 2/50, Valence Train Loss: 0.1332, Arousal Train Loss: 0.1339
Epoch 2/50, Valence Test Loss : 0.1394, Arousal Test Loss: 0.1471
Epoch 2/50, Valence CCC Loss : 0.9556, Arousal CCC Loss: 1.0089
Epoch 3/50, Valence Train Loss: 0.1076, Arousal Train Loss: 0.1268
Epoch 3/50, Valence Test Loss : 0.1281, Arousal Test Loss: 0.0950
Epoch 3/50, Valence CCC Loss : 0.8977, Arousal CCC Loss: 1.0128
Epoch 4/50, Valence Train Loss: 0.0847, Arousal Train Loss: 0.1130
Epoch 4/50, Valence Test Loss : 0.0844, Arousal Test Loss: 0.1126
Epoch 4/50, Valence CCC Loss : 0.8283, Arousal CCC Loss: 0.9839
Epoch 5/50, Valence Train Loss: 0.0729, Arousal Train Loss: 0.0958
Epoch 5/50, Valence Test Loss : 0.0785, Arousal Test Loss: 0.1035
Epoch 5/50, Valence CCC Loss : 0.8143, Arousal CCC Loss: 0.9435
Epoch 6/50, Val

In [ ]:
#training on c1 and c4 testing on all others

In [54]:
tensors, valence, arousal = get_data(df_C4_video)
normalized_feature_tensors = normalize(tensors)

X_train, X_test = train_test_split(normalized_feature_tensors, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

y_valence = np.array(valence)
y_arousal = np.array(arousal)

y_train_valence, y_test_valence, y_train_arousal, y_test_arousal = train_test_split(y_valence, y_arousal, test_size=0.2, random_state=42)
print("y_train_valence shape:", y_train_valence.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

X_train shape: (47, 750, 512)
X_test shape: (12, 750, 512)
y_train_valence shape: (47, 750)
y_test_valence shape: (12, 750)
y_train_arousal shape: (47, 750)
y_test_arousal shape: (12, 750)


In [97]:
def normalize(features, max_length=750):
    padded_features = pad_sequences(features, maxlen=max_length, padding='post', dtype='float32')
    normalized_feature_tensors = []
    masks = []  # 
    for video_features in padded_features:
        # Compute mean and standard deviation along the first axis (frames)
        mean = np.mean(video_features, axis=0, keepdims=True)
        std = np.std(video_features, axis=0, keepdims=True)
        # Apply normalization to each frame
        normalized_video_features = (video_features - mean) / (std + 1e-8)  # Adding a small epsilon to avoid division by zero
        normalized_feature_tensors.append(normalized_video_features)
        # 
        mask = np.ones_like(video_features)  # 
        mask[len(video_features):] = 0  # 
        masks.append(mask)

    # Convert the list of normalized feature tensors back to a numpy array
    normalized_feature_tensors = np.array(normalized_feature_tensors)
    masks = np.array(masks)

    return normalized_feature_tensors, masks

In [99]:
print("testing c2 training on c1 combined c4")
tensors_C4, valence_C4, arousal_C4 = get_data(df_C4_video)
normalized_feature_tensors_C4 = []
normalized_feature_tensors_C4 = normalize(tensors_C4)

print(len(normalized_feature_tensors_C4_))
print(len(valence_C4))
print(len(arousal_C4))

testing c2 training on c1 combined c4
59
59
59


In [100]:
tensors_C1=[]
valence_C1 =[]
arousal_C1 =[]

tensors_C1, valence_C1, arousal_C1 = get_data(df_C1_video)
normalized_feature_tensors_C1 =[] 
normalized_feature_tensors_C1 = normalize(tensors_C1)

print(len(normalized_feature_tensors_C1))
print(len(tensors_C1))
print(len(valence_C1))
print(len(arousal_C1))

75
75
75
75


In [110]:

X_train= np.concatenate((normalized_feature_tensors_C1, normalized_feature_tensors_C4), axis=0)
y_train_valence = np.concatenate((valence_C1, valence_C4), axis=0)
y_train_arousal = np.concatenate((arousal_C1, arousal_C4), axis=0)


In [111]:

print("testing c2 training on c1 combined c4")
tensors_C2, valence_C2, arousal_C2 = get_data(df_C2_video)
print(len(tensors_C2))
normalized_test_feature_tensors_C2 = normalize(tensors_C2)
print(len(normalized_test_feature_tensors_C2))
X_test = normalized_test_feature_tensors_C2
y_test_valence = np.array(valence_C2)
y_test_arousal = np.array(arousal_C2)
print("X_train shape:", X_train.shape)
print("y_train_valence shape:", y_train_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)

print("X_test shape:", X_test.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

81
81
X_train shape: (134, 750, 512)
y_train_valence shape: (134, 750)
y_train_arousal shape: (134, 750)
X_test shape: (81, 750, 512)
y_test_valence shape: (81, 750)
y_test_arousal shape: (81, 750)


In [109]:
RNN_model(X_train, X_test)

Epoch 1/5, Valence Train Loss: 0.1754, Arousal Train Loss: 0.1518
Epoch 1/5, Valence Test Loss: 0.2991, Arousal Test Loss: 0.3356
Epoch 2/5, Valence Train Loss: 0.1694, Arousal Train Loss: 0.1479
Epoch 2/5, Valence Test Loss: 0.2954, Arousal Test Loss: 0.3319
Epoch 3/5, Valence Train Loss: 0.1659, Arousal Train Loss: 0.1450
Epoch 3/5, Valence Test Loss: 0.2917, Arousal Test Loss: 0.3282
Epoch 4/5, Valence Train Loss: 0.1626, Arousal Train Loss: 0.1429
Epoch 4/5, Valence Test Loss: 0.2877, Arousal Test Loss: 0.3244
Epoch 5/5, Valence Train Loss: 0.1590, Arousal Train Loss: 0.1409
Epoch 5/5, Valence Test Loss: 0.2834, Arousal Test Loss: 0.3200


In [ ]:
LSTM_model(X_train, X_test)

In [ ]:
LSTM_Attention_Model(X_train, X_test)

In [ ]:
bi_GRU_model(X_train, X_test)

In [113]:
print("testing c3 training on c1 combined c4")
tensors_C3, valence_C3, arousal_C3 = get_data(df_C3_video)
normalized_test_feature_tensors_C3 = normalize(tensors_C3)
X_test = normalized_test_feature_tensors_C3
y_test_valence = np.array(valence_C3)
y_test_arousal = np.array(arousal_C3)
print("X_train shape:", X_train.shape)
print("y_train_valence shape:", y_train_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)

print("X_test shape:", X_test.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_test_arousal shape:", y_test_arousal.shape)


testing c3 training on c1 combined c4
X_train shape: (134, 750, 512)
y_train_valence shape: (134, 750)
y_train_arousal shape: (134, 750)
X_test shape: (72, 750, 512)
y_test_valence shape: (72, 750)
y_test_arousal shape: (72, 750)


In [ ]:
RNN_model(X_train, X_test)

In [ ]:
LSTM_model(X_train, X_test)

In [ ]:
LSTM_Attention_Model(X_train, X_test)

In [ ]:
bi_GRU_model(X_train, X_test)

In [115]:
print("testing c5 training on c1 combined c4")
tensors_C5, valence_C5, arousal_C5 = get_data(df_C5_video)
normalized_test_feature_tensors_C5 = normalize(tensors_C5)
X_test = normalized_test_feature_tensors_C5
y_test_valence = np.array(valence_C5)
y_test_arousal = np.array(arousal_C5)
print("X_train shape:", X_train.shape)
print("y_train_valence shape:", y_train_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)

print("X_test shape:", X_test.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

testing c5 training on c1 combined c4
X_train shape: (134, 750, 512)
y_train_valence shape: (134, 750)
y_train_arousal shape: (134, 750)
X_test shape: (60, 750, 512)
y_test_valence shape: (60, 750)
y_test_arousal shape: (60, 750)


In [ ]:
RNN_model(X_train, X_test)

In [ ]:
LSTM_model(X_train, X_test)

In [ ]:
LSTM_Attention_Model(X_train, X_test)

In [ ]:
bi_GRU_model(X_train, X_test)

In [117]:
print("testing c6 training on c1 combined c4")
tensors_C6, valence_C6, arousal_C6 = get_data(df_C6_video)
normalized_test_feature_tensors_C6 = normalize(tensors_C6)
X_test = normalized_test_feature_tensors_C6
y_test_valence = np.array(valence_C6)
y_test_arousal = np.array(arousal_C6)
print("X_train shape:", X_train.shape)
print("y_train_valence shape:", y_train_valence.shape)
print("y_train_arousal shape:", y_train_arousal.shape)

print("X_test shape:", X_test.shape)
print("y_test_valence shape:", y_test_valence.shape)
print("y_test_arousal shape:", y_test_arousal.shape)

testing c6 training on c1 combined c4
X_train shape: (134, 750, 512)
y_train_valence shape: (134, 750)
y_train_arousal shape: (134, 750)
X_test shape: (69, 750, 512)
y_test_valence shape: (69, 750)
y_test_arousal shape: (69, 750)


In [ ]:
RNN_model(X_train, X_test)

In [ ]:
LSTM_model(X_train, X_test)

In [ ]:
LSTM_Attention_Model(X_train, X_test)

In [ ]:
bi_GRU_model(X_train, X_test)